In [1]:
"""
    交叉特征
    
"""

'\n    交叉特征\n    \n'

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.17.4
pandas 0.25.3
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [3]:
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

y_train = train_df.pop("survived")
y_eval =  eval_df.pop("survived")


In [4]:
# feature_column
categorical_columns = ["sex", "n_siblings_spouses", "parch", 
                       "class", "deck", "embark_town", "alone"]
numeric_columns = ["age","fare" ]
feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column, vocab)
    # 离散特征作成one-hot编码
    onehot_describe = tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            categorical_column, vocab))
    feature_columns.append(onehot_describe)
    
for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            numeric_column, dtype = tf.float32))
    
# cross_feature: 
# age_x_gender: [(1, male),(2, male), .....(5, male), (1,female), ....]

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


#### 生成交叉特征

In [5]:
cross_describe = tf.feature_column.indicator_column(
    tf.feature_column.crossed_column(["age", "sex"],
       hash_bucket_size = 100))
feature_columns.append(cross_describe)

In [6]:
def make_dataset(data_df, label_df, epochs = 10,# 数据集遍历多少次 ， 
                 shuffle = True, batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices(
         (dict(data_df),label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [7]:
output_dir = "baseline_model_new_feature"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
baseline_estimator = tf.compat.v1.estimator.BaselineClassifier(
    model_dir=output_dir,
    n_classes=2)
# baseline_estimator = tf.estimator.BaselineClassifier(
#     model_dir = output_dir, 
#     n_class = 2)
baseline_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs = 100))
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'baseline_model_new_feature', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000188F4AA7048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras 

In [8]:
baseline_estimator.evaluate(input_fn = lambda: make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False, batch_size = 20))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-07T23:31:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from baseline_model_new_feature\model.ckpt-3920
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-07-23:31:03
INFO:tensorflow:Saving dict for global step 3920: accuracy = 0.625, accuracy_baseline = 0.625, auc = 0.5, auc_precision_recall = 0.6875, average_loss = 0.66189986, global_step = 3920, label/mean = 0.375, loss = 12.48154, precision = 0.0, prediction/mean = 0.38761556, recall = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3920: baseline_model_new_feature\model.ckpt-3920


{'accuracy': 0.625,
 'accuracy_baseline': 0.625,
 'auc': 0.5,
 'auc_precision_recall': 0.6875,
 'average_loss': 0.66189986,
 'global_step': 3920,
 'label/mean': 0.375,
 'loss': 12.48154,
 'precision': 0.0,
 'prediction/mean': 0.38761556,
 'recall': 0.0}

In [9]:
linear_output_dir = "linear_model_new_feature"
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)
linear_estimator = tf.estimator.LinearClassifier(
    model_dir=linear_output_dir,
    n_classes=2,
    feature_columns = feature_columns)
linear_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'linear_model_new_feature', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000188F639EF28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you in

In [10]:
 linear_estimator.evaluate(input_fn = lambda: make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-07T23:31:24Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_model_new_feature\model.ckpt-3920
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-07-23:31:25
INFO:tensorflow:Saving dict for global step 3920: accuracy = 0.8030303, accuracy_baseline = 0.625, auc = 0.854698

{'accuracy': 0.8030303,
 'accuracy_baseline': 0.625,
 'auc': 0.85469854,
 'auc_precision_recall': 0.76358724,
 'average_loss': 0.47868976,
 'global_step': 3920,
 'label/mean': 0.375,
 'loss': 0.46008697,
 'precision': 0.7196262,
 'prediction/mean': 0.43416768,
 'recall': 0.7777778}

In [11]:
dnn_output_dir = "./dnn_model_new_feature"
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(
    model_dir=dnn_output_dir,
    n_classes=2,
    feature_columns = feature_columns,
    hidden_units = [128,128],
    activation_fn = tf.nn.relu,
    optimizer = "Adam")
dnn_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './dnn_model_new_feature', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000188F966A390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you int

In [12]:
dnn_estimator.evaluate(input_fn = lambda: make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-07T23:31:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./dnn_model_new_feature\model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-07-23:31:34
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.74242425, accuracy_baseline = 0.625, auc = 0.814998

{'accuracy': 0.74242425,
 'accuracy_baseline': 0.625,
 'auc': 0.81499845,
 'auc_precision_recall': 0.7687728,
 'average_loss': 0.672304,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.65431356,
 'precision': 0.65346533,
 'prediction/mean': 0.3930917,
 'recall': 0.6666667}